In [1]:
import numpy as np
from scipy.integrate import fixed_quad
import plotly.graph_objects as go
import pandas as pd
from iminuit import Minuit
from iminuit.cost import LeastSquares
import matplotlib.pyplot as plt
import os 
import time 
import scipy.stats as stats

In [2]:
save_folder = 'run5'
n_points = 10000

lower_factor = 0.99
upper_factor = 2 - lower_factor

In [3]:
# Load experimental data
atlas_data = pd.read_csv('../../data/ens_atlas_difc0_2.dat', delim_whitespace=True, header=None)
totem_data = pd.read_csv('../../data/ens_totem_difc0_2.dat', delim_whitespace=True, header=None)

# Function to process data for each experiment
def process_data(data, energy_blocks):
    x_values = []
    y_values = []
    y_errors = []
    
    for start, end in energy_blocks:
        block = data.iloc[start:end] if end is not None else data.iloc[start:]
        x_values.append(block[0].values)
        y_values.append(block[1].values)
        y_errors.append(block[2].values)
    
    return x_values, y_values, y_errors

# Energy ranges for each experiment (7TeV, 8TeV, 13TeV)
atlas_blocks = [(0, 29), (29, 58), (58, None)]
totem_blocks = [(0, 65), (65, 118), (118, None)]

# Process data
x_atlas, y_atlas, yerr_atlas = process_data(atlas_data, atlas_blocks)
x_totem, y_totem, yerr_totem = process_data(totem_data, totem_blocks)

# Extract values by energy (index 0=7TeV, 1=8TeV, 2=13TeV)
x_7_atlas, y_7_atlas, yerr_7_atlas = x_atlas[0], y_atlas[0], yerr_atlas[0]
x_8_atlas, y_8_atlas, yerr_8_atlas = x_atlas[1], y_atlas[1], yerr_atlas[1]
x_13_atlas, y_13_atlas, yerr_13_atlas = x_atlas[2], y_atlas[2], yerr_atlas[2]

x_7_totem, y_7_totem, yerr_7_totem = x_totem[0], y_totem[0], yerr_totem[0]
x_8_totem, y_8_totem, yerr_8_totem = x_totem[1], y_totem[1], yerr_totem[1]
x_13_totem, y_13_totem, yerr_13_totem = x_totem[2], y_totem[2], yerr_totem[2]

/tmp/ipykernel_38374/1917071991.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  atlas_data = pd.read_csv('../../data/ens_atlas_difc0_2.dat', delim_whitespace=True, header=None)
/tmp/ipykernel_38374/1917071991.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  totem_data = pd.read_csv('../../data/ens_totem_difc0_2.dat', delim_whitespace=True, header=None)


In [4]:

b_0 = (33 - 6) / (12 * np.pi)
Lambda = 0.284  # ΛQCD in GeV
gamma_1 = 0.084
gamma_2 = 2.36
rho = 4.0
s0 = 1.0
alpha_prime = 0.25



ensemble_parameters = {
    'atlas': {
        'log': {
            'epsilon': 0.0753,
            'mg': 0.356,
            'a1': 1.373,
            'a2': 2.50
        },
        'pl': {
            'epsilon': 0.0753,
            'mg': 0.421,
            'a1': 1.517,
            'a2': 2.05
        }
    },
    'totem': {
        'log': {
            'epsilon': 0.0892,
            'mg': 0.380,
            'a1': 1.491,
            'a2': 2.77
        },
        'pl':{
            'epsilon': 0.0892,
            'mg': 0.447,
            'a1': 1.689,
            'a2': 1.7
        }
    }
}

ensemble_atlas = 'atlas'  
ensemble_totem = 'totem'

log_model_type = 'log'
pl_model_type = 'pl'   

def get_parameters_with_variations(ensemble_parameters, ensemble_name, model_type, lower_factor=lower_factor, upper_factor=upper_factor):
    # Obtém os parâmetros iniciais
    initial_params = ensemble_parameters[ensemble_name][model_type]
    
    # Cria as variações
    initial_params_low = {k: v * lower_factor for k, v in initial_params.items()}
    initial_params_high = {k: v * upper_factor for k, v in initial_params.items()}
    
    return initial_params, initial_params_low, initial_params_high

# Get parameters for selected configuration
initial_params_log_atlas = ensemble_parameters[ensemble_atlas][log_model_type]
initial_params_pl_atlas = ensemble_parameters[ensemble_atlas][pl_model_type]

# Para Atlas
initial_params_log_atlas, initial_params_low_log_atlas, initial_params_high_log_atlas = \
    get_parameters_with_variations(ensemble_parameters, ensemble_atlas, log_model_type)

initial_params_pl_atlas, initial_params_low_pl_atlas, initial_params_high_pl_atlas = \
    get_parameters_with_variations(ensemble_parameters, ensemble_atlas, pl_model_type)

# Para Totem
initial_params_log_totem, initial_params_low_log_totem, initial_params_high_log_totem = \
    get_parameters_with_variations(ensemble_parameters, ensemble_totem, log_model_type)

initial_params_pl_totem, initial_params_low_pl_totem, initial_params_high_pl_totem = \
    get_parameters_with_variations(ensemble_parameters, ensemble_totem, pl_model_type)



In [5]:
def m2_log(q2, mg):
    lambda_squared = Lambda ** 2
    rho_mg_squared = rho * mg ** 2
    ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
    return mg ** 2 * ratio ** (-1 - gamma_1)

def m2_pl(q2, mg):
    lambda_squared = Lambda ** 2
    rho_mg_squared = rho * mg ** 2
    ratio = np.log((q2 + rho_mg_squared) / lambda_squared) / np.log(rho_mg_squared / lambda_squared)
    return (mg ** 4 / (q2 + mg ** 2)) * ratio ** (gamma_2 - 1)


def G_p(q2, a1, a2):
    return np.exp(-(a1 * q2 + a2 * q2 ** 2))

def alpha_D(q2, mg, m2_func):
    m2 = m2_func(q2, mg)
    return 1.0 / (b_0 * (q2 + m2) * np.log((q2 + 4 * m2) / (Lambda ** 2)))

def T_1(k, q, phi, mg, a1, a2, m2_func):
    q2 = q 
    qk_cos = np.sqrt(q) * k * np.cos(phi)
    qk_plus_squared = q2 / 4 + qk_cos + k ** 2
    qk_minus_squared = q2 / 4 - qk_cos + k ** 2
    alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
    alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
    G0 = G_p(q2, a1, a2)
    return alpha_D_plus * alpha_D_minus * G0 ** 2

def T_2(k, q, phi, mg, a1, a2, m2_func):
    q2 = q 
    qk_cos = np.sqrt(q) * k * np.cos(phi)
    qk_plus_squared = q2 / 4 + qk_cos + k ** 2
    qk_minus_squared = q2 / 4 - qk_cos + k ** 2
    alpha_D_plus = alpha_D(qk_plus_squared, mg, m2_func)
    alpha_D_minus = alpha_D(qk_minus_squared, mg, m2_func)
    factor = q2 + 9 * abs(k ** 2 - q2 / 4)
    G0 = G_p(q2, a1, a2)
    G_minus = G_p(factor, a1, a2)
    return alpha_D_plus * alpha_D_minus * G_minus * (2 * G0 - G_minus)

def integrand(y, x, mg, a1, a2, m2_func, q_val, sqrt_s):
    k = sqrt_s * x 
    phi = 2 * np.pi * y
    jacobian = 2 * np.pi * sqrt_s 
    return k * (T_1(k, q_val, phi, mg, a1, a2, m2_func) - T_2(k, q_val, phi, mg, a1, a2, m2_func)) * jacobian 

def sigma_tot(amp_value, s):
    return amp_value.imag / s * 0.389379323

def amp_calculation(diff_T, s, epsilon, t):
    alpha_pomeron = 1.0 + epsilon + alpha_prime * t
    regge_factor = (s**alpha_pomeron) * 1/(s0**(alpha_pomeron-1))
    return 1j * 8 * regge_factor * diff_T  

def differential_sigma(amp_value, s):
    amp_squared = amp_value.imag * amp_value.imag
    denominator = (16 * np.pi * s**2)
    return amp_squared / denominator * 0.389379323


In [6]:
def model_function(x, eps, mg, a1, a2, sqrt_s, model_type='log'):

    # Definindo os parâmetros específicos do modelo
    params = {
        'epsilon': eps,
        'mg': mg,
        'a1': a1,
        'a2': a2
    }
    
    # Escolhendo a massa conforme o modelo
    m2 = m2_log if model_type == 'log' else m2_pl
    
    dif_sigma_lst = []
    
    for q2 in x:
        t = -q2
        
        def inner_integral(x_inner):
            return fixed_quad(
                lambda y: integrand(y, x_inner, params['mg'], params['a1'], 
                                  params['a2'], m2, q2, sqrt_s),
                0, 1,
                n=n_points
            )[0]

        integral_value = fixed_quad(
            inner_integral,
            0, 1,
            n=n_points
        )[0]

        diff_T = integral_value
        s = sqrt_s ** 2
        amp_value = amp_calculation(diff_T, s, params['epsilon'], t)
        dif_sigma_value = differential_sigma(amp_value, s)
        dif_sigma_lst.append(dif_sigma_value)
    
    return np.array(dif_sigma_lst)

In [7]:
def create_least_squares(x, y, yerr, sqrt_s, model_type):
    return LeastSquares(x, y, yerr, 
        lambda x, eps, mg, a1, a2: model_function(x, eps, mg, a1, a2, sqrt_s, model_type))

def total_cost(dataset, model_type):
    return sum([
        create_least_squares(*dataset[7000], 7000, model_type),
        create_least_squares(*dataset[8000], 8000, model_type),
        create_least_squares(*dataset[13000], 13000, model_type)
    ])

# Estrutura dos dados por experimento
atlas_data = {
    7000: (x_7_atlas, y_7_atlas, yerr_7_atlas),
    8000: (x_8_atlas, y_8_atlas, yerr_8_atlas),
    13000: (x_13_atlas, y_13_atlas, yerr_13_atlas)
}

totem_data = {
    7000: (x_7_totem, y_7_totem, yerr_7_totem),
    8000: (x_8_totem, y_8_totem, yerr_8_totem),
    13000: (x_13_totem, y_13_totem, yerr_13_totem)
}

# Custos totais
total_cost_log_atlas = total_cost(atlas_data, 'log')
total_cost_pl_atlas = total_cost(atlas_data, 'pl')
total_cost_log_totem = total_cost(totem_data, 'log')
total_cost_pl_totem = total_cost(totem_data, 'pl')


In [8]:
# def otimization(total_cost_func, initial_params, initial_params_low, initial_params_high, model_type: str, ensemble: str):
#     print('\n')
#     print(80 * '-')
#     print(f"Iniciando otimização dos parâmetros usando LeastSquares para {model_type} em {ensemble.upper()}")

#     start_time = time.time()
    
#     m = Minuit(total_cost_func, 
#                eps=initial_params['epsilon'],
#                mg=initial_params['mg'],
#                a1=initial_params['a1'],
#                a2=initial_params['a2'])

#     if model_type == 'pl':
#         # Configurações adicionais
#         down = 0.97
#         up = 2- down

#         m.limits['mg'] = (down * initial_params['mg'], up *initial_params['mg'])
#         m.limits['eps'] = (down * initial_params['epsilon'], up *initial_params['epsilon'])
#         # m.limits['a2'] = (initial_params_low['a2'], initial_params_high['a2'])

#         m.limits['a1'] = (initial_params_low['a1'], initial_params_high['a1'])

#         # Configurações adicionais
#         m.strategy = 2
#         m.errordef = 7.79
#         m.tol = 1e-2
        
#         m.migrad(ncall=250)
#         m.hesse(ncall=20)

#     else:
#         down = 0.99
#         up = 2- down

#         m.limits['mg'] = (down * initial_params['mg'], up * initial_params['mg'])
#         m.limits['eps'] = (down * initial_params['epsilon'], up * initial_params['epsilon'])
#         m.limits['a2'] = (down * initial_params['a2'], up * initial_params['a2'])
#         # m.limits['a1'] = (down * initial_params['a1'], up * initial_params['a1'])

#         # Configurações adicionais
#         m.strategy = 2
#         m.errordef = 7.79
#         m.tol = 1e-2
        
#         m.migrad(ncall=90)
#         m.hesse(ncall=10)
            

#     print(f'Finalizado a minimização para {model_type} em {ensemble}')

#     execution_time = time.time() - start_time
    
#     minutes = int(execution_time // 60)
#     seconds = execution_time % 60
#     print(f'Tempo de execução para {model_type} em {ensemble}: {minutes} min {seconds:.2f} s \n')
    
#     print(f"Parâmetros otimizados para {model_type} em {ensemble}: \n")
#     print(f'mg: {m.values["mg"]} ± {m.errors["mg"]}')
#     print(f'eps: {m.values["eps"]} ± {m.errors["eps"]}')
#     print(f'a1: {m.values["a1"]} ± {m.errors["a1"]}')
#     print(f'a2: {m.values["a2"]} ± {m.errors["a2"]}')
#     print(f'chi2/ndof: {m.fval / m.ndof}')

    

#     # for par in m.parameters:
#     #     e = m.merrors[par]
#     #     print(f"{par} = {m.values[par]} +{e.upper} -{e.lower}")


#     return m

# '''
# perct = 0.99
# str = 2
# errordef = 7.779
# tol = 1e-3
# migrad = 250
# hesse = 20
# '''




# m_log_atlas = otimization(total_cost_log_atlas,
#                           initial_params_log_atlas,
#                           initial_params_low_log_atlas,
#                           initial_params_high_log_atlas, 'log', 'atlas')

# # m_pl_atlas = otimization(total_cost_pl_atlas,
# #                          initial_params_pl_atlas,
# #                          initial_params_low_pl_atlas,
# #                          initial_params_high_pl_atlas, 'pl', 'atlas')

# m_log_totem = otimization(total_cost_log_totem,
#                           initial_params_log_totem,
#                           initial_params_low_log_totem,
#                           initial_params_high_log_totem, 'log', 'totem')

# # m_pl_totem = otimization(total_cost_pl_totem,
# #                          initial_params_pl_totem,
# #                          initial_params_low_pl_totem,
# #                          initial_params_high_pl_totem, 'pl', 'totem')


# # 0.97 aplicado a mg e eps 
# # mg: 0.41554072054223007 ± 0.0011974110056826859
# # eps: 0.07307497583527327 ± 0.0026838167524921436
# # chi2/ndof: 0.49346399961109705



In [9]:
import itertools

def otimization(total_cost_func, initial_params, initial_params_low, initial_params_high, model_type: str, ensemble: str):
    print(f"\nIniciando otimização para {model_type} em {ensemble.upper()}\n")
    
    # Valores a serem testados
    down_values = [0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1]  
    strategy_values = [0, 1, 2]
    migrad_calls = [60, 65, 70, 75, 80, 85, 90, 95, 100, 150, 200]
    
    # Gerar todas as 8 combinações possíveis de parâmetros limitados
    limit_combinations = list(itertools.product([False, True], repeat=3))  # 3 parâmetros: eps, mg, a2
    
    for limit_eps, limit_mg, limit_a2 in limit_combinations:
        # Criar nome do arquivo baseado na combinação de limites
        limit_str = f"eps_{int(limit_eps)}_mg_{int(limit_mg)}_a2_{int(limit_a2)}"
        output_filename = f"resultados_otimizacao_{model_type}_{ensemble}_{limit_str}.txt"
        
        # Cabeçalho do arquivo
        with open(output_filename, 'w') as f:
            f.write(f"Otimização para {model_type} em {ensemble.upper()} - Limites: {limit_str}\n")
            f.write("Configuração: down | up | strategy | migrad | mg ± error | eps ± error | a1 ± error | a2 ± error | χ²/ndof\n")
            f.write("="*120 + "\n")
        
        for down in down_values:
            up = 2 - down
            for strategy in strategy_values:
                for ncall in migrad_calls:
                    config_str = f"\n=== Configuração: down={down}, up={up}, strategy={strategy}, migrad={ncall} ==="
                    print(config_str)
                    
                    try:
                        m = Minuit(total_cost_func, 
                                 eps=initial_params['epsilon'],
                                 mg=initial_params['mg'],
                                 a1=initial_params['a1'],
                                 a2=initial_params['a2'])
                        
                        # Aplicar limites conforme a combinação atual
                        if limit_a2:
                            m.limits['a2'] = (down * initial_params['a2'], up * initial_params['a2'])
                        if limit_mg:
                            m.limits['mg'] = (down * initial_params['mg'], up * initial_params['mg'])
                        if limit_eps:
                            m.limits['eps'] = (down * initial_params['epsilon'], up * initial_params['epsilon'])

                        m.strategy = strategy
                        m.errordef = 7.79
                        m.tol = 1e-2
                        
                        m.migrad(ncall=ncall)
                        m.hesse(ncall=10)

                        # Escrever no arquivo específico desta combinação de limites
                        if m.ndof > 0:
                            chi2_ndof = m.fval / m.ndof
                            if 0.49 <= chi2_ndof <= 1.1:
                                with open(output_filename, 'a') as f:
                                    f.write(f"{down} | {up} | {strategy} | {ncall} | "
                                            f"{m.values['mg']} ± {m.errors['mg']} | "
                                            f"{m.values['eps']} ± {m.errors['eps']} | "
                                            f"{m.values['a1']} ± {m.errors['a1']} | "
                                            f"{m.values['a2']} ± {m.errors['a2']} | "
                                            f"{m.fval/m.ndof}\n")
                        
                    except Exception as e:
                        error_msg = f"Falha na execução: {str(e)}"
                        print(error_msg)
                        with open(output_filename, 'a') as f:
                            f.write(f"{down} | {up} | {strategy} | {ncall} | "
                                   f"FALHA: {str(e)}\n")
                        continue

        print(f"\nConcluído combinação de limites {limit_str}. Resultados salvos em {output_filename}")

    print("\nConcluído todas as combinações de parâmetros.")
    return None

In [10]:
m_log_totem = otimization(total_cost_log_totem,
                          initial_params_log_totem,
                          initial_params_low_log_totem,
                          initial_params_high_log_totem, 'log', 'totem')


/home/victorli/miniconda3/lib/python3.12/site-packages/iminuit/minuit.py:139: ErrordefAlreadySetWarning: cost function has an errordef attribute equal to 1.0, you should not override this with Minuit.errordef
  warnings.warn(msg, ErrordefAlreadySetWarning)



Iniciando otimização para log em TOTEM


=== Configuração: down=0.91, up=1.0899999999999999, strategy=0, migrad=60 ===

=== Configuração: down=0.91, up=1.0899999999999999, strategy=0, migrad=65 ===

=== Configuração: down=0.91, up=1.0899999999999999, strategy=0, migrad=70 ===

=== Configuração: down=0.91, up=1.0899999999999999, strategy=0, migrad=75 ===


KeyboardInterrupt: 

In [ ]:

m_log_atlas = otimization(total_cost_log_atlas,
                          initial_params_log_atlas,
                          initial_params_low_log_atlas,
                          initial_params_high_log_atlas, 'log', 'atlas')


In [ ]:
# def add_data_trace(fig, x, y, y_error, scale=1.0, color='black', size=4,
#                            name=None, show_label=True, mode='markers'):
#     fig.add_trace(go.Scatter(
#         x=x,
#         y=y * scale,
#         mode=mode,
#         marker=dict(color=color, size=size),
#         error_y=dict(
#             type='data',
#             array=y_error * scale,
#             visible=True
#         ),
#         name=name if show_label else None,
#         showlegend=show_label
#     ))


# data_sigma_tot_atlas = pd.read_csv(
#     "../../data/sigma_tot_2/ensemble_StRh_atlas.dat",
#     delim_whitespace=True,
#     header=None,
#     nrows=70
# )

# data_sigma_tot_totem = pd.read_csv(
#     "../../data/sigma_tot_2/ensemble_StRh_totem.dat",  # Supondo que existe um arquivo similar para TOTEM
#     delim_whitespace=True,
#     header=None,
#     nrows=80
# )

# x_sigma_tot_atlas = data_sigma_tot_atlas[0].to_numpy()
# y_sigma_tot_atlas = data_sigma_tot_atlas[1].to_numpy()
# y_error_sigma_tot_atlas = data_sigma_tot_atlas[2].to_numpy()

# x_sigma_tot_totem = data_sigma_tot_totem[0].to_numpy()
# y_sigma_tot_totem = data_sigma_tot_totem[1].to_numpy()
# y_error_sigma_tot_totem = data_sigma_tot_totem[2].to_numpy()

# start_sqrt_s = 1
# max_sqrt_s = 13010
# step = 100

# def add_total_trace(fig, x, y, color='red', label='', line_style='solid', legend=True, size = 3, width = 2):
#     fig.add_trace(go.Scatter(
#         x=x,
#         y=y,
#         mode='lines+markers',
#         line=dict(color=color, width=width, dash=line_style),
#         marker=dict(size=size),
#         name = label, 
#         showlegend=legend
#     ))

# def get_sigma_tot(epsilon, mg, a1, a2, mg_model):

#     lst_sigma_tot = []
#     lst_sqrt_s = []

#     sqrt_s = start_sqrt_s

#     while sqrt_s <= max_sqrt_s:
#         def integrand(y, x, mg, a1, a2, m2_func):
#             k = sqrt_s * x
#             phi = 2 * np.pi * y
#             jacobian = 2 * np.pi * sqrt_s

#             return k * (T_1(k, 0.0, phi, mg, a1, a2, m2_func) - T_2(k, 0.0, phi, mg, a1, a2, m2_func)) * jacobian
        
#         def amp_calculation(diff_T, s, epsilon):
#             alpha_pomeron = 1.0 + epsilon
#             regge_factor = (s / s0) ** alpha_pomeron
            
#             return 1j * 8.0 * regge_factor * diff_T

#         s = sqrt_s ** 2

#         integral_value = fixed_quad(
#             lambda x: fixed_quad(
#                 lambda y: integrand(y, x, mg, a1, a2, mg_model),
#                 0, 1, n=n_points)[0],

#             0, 1, n=n_points)[0]
        
#         lst_sigma_tot.append(sigma_tot(
#             amp_calculation(integral_value, s, epsilon), s))
        
#         lst_sqrt_s.append(sqrt_s)
#         sqrt_s += step
#     return lst_sigma_tot, lst_sqrt_s
# #-----------------------------------------------------------------------------------------------
# sigma_tot_log_atlas_paper = get_sigma_tot(
#     ensemble_parameters['atlas']['log']['epsilon'],
#     ensemble_parameters['atlas']['log']['mg'],
#     ensemble_parameters['atlas']['log']['a1'],
#     ensemble_parameters['atlas']['log']['a2'],
#     m2_log
# )

# sigma_tot_pl_atlas_paper = get_sigma_tot(
#     ensemble_parameters['atlas']['pl']['epsilon'],
#     ensemble_parameters['atlas']['pl']['mg'],
#     ensemble_parameters['atlas']['pl']['a1'],
#     ensemble_parameters['atlas']['pl']['a2'],
#     m2_pl
# )
# sigma_tot_log_totem_paper = get_sigma_tot(
#     ensemble_parameters['totem']['log']['epsilon'],
#     ensemble_parameters['totem']['log']['mg'],
#     ensemble_parameters['totem']['log']['a1'],
#     ensemble_parameters['totem']['log']['a2'],
#     m2_log
# )

# sigma_tot_pl_totem_paper = get_sigma_tot(
#     ensemble_parameters['totem']['pl']['epsilon'],
#     ensemble_parameters['totem']['pl']['mg'],
#     ensemble_parameters['totem']['pl']['a1'],
#     ensemble_parameters['totem']['pl']['a2'],
#     m2_pl
# )


# lst_sqrt_s = sigma_tot_log_atlas_paper[1]

# sigma_tot_log_atlas_values_paper = sigma_tot_log_atlas_paper[0]
# sigma_tot_pl_atlas_values_paper = sigma_tot_pl_atlas_paper[0]
# sigma_tot_log_totem_values_paper = sigma_tot_log_totem_paper[0]
# sigma_tot_pl_totem_values_paper = sigma_tot_pl_totem_paper[0]

# #-----------------------------------------------------------------------------------------------

# sigma_tot_log_atlas = get_sigma_tot(
#     m_log_atlas.values['eps'],
#     m_log_atlas.values['mg'],
#     m_log_atlas.values['a1'],
#     m_log_atlas.values['a2'],
#     m2_log
# )

# # sigma_tot_pl_atlas = get_sigma_tot(
# #     m_pl_atlas.values['eps'],
# #     m_pl_atlas.values['mg'],
# #     m_pl_atlas.values['a1'],
# #     m_pl_atlas.values['a2'],
# #     m2_pl
# # )

# sigma_tot_log_totem = get_sigma_tot(
#     m_log_totem.values['eps'],
#     m_log_totem.values['mg'],
#     m_log_totem.values['a1'],
#     m_log_totem.values['a2'],
#     m2_log
# )

# # sigma_tot_pl_totem = get_sigma_tot(
# #     m_pl_totem.values['eps'],
# #     m_pl_totem.values['mg'],
# #     m_pl_totem.values['a1'],
# #     m_pl_totem.values['a2'],
# #     m2_pl
# # )

# lst_sqrt_s = sigma_tot_log_atlas[1]


# sigma_tot_log_atlas_values = sigma_tot_log_atlas[0]
# # sigma_tot_pl_atlas_values = sigma_tot_pl_atlas[0]

# sigma_tot_log_totem_values = sigma_tot_log_totem[0]
# # sigma_tot_pl_totem_values = sigma_tot_pl_totem[0]

# fig = go.Figure()

# add_total_trace(fig, lst_sqrt_s, sigma_tot_log_atlas_values, color='red', label='Log Atlas', line_style='solid')
# # add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_atlas_values, color='blue', label='PL Atlas', line_style='solid')

# add_total_trace(fig, lst_sqrt_s, sigma_tot_log_totem_values, color='red', label='Log Totem', line_style='dot')
# # add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_totem_values, color='blue', label='PL Totem', line_style='dot')

# #-----------------------------------------------------------------------------------------------
# add_total_trace(fig, lst_sqrt_s, sigma_tot_log_atlas_values_paper, color='gray', label='Log Atlas (Paper)', line_style='solid', legend=True, size = 2, width=1)
# add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_atlas_values_paper, color='gray', label='PL Atlas (Paper)', line_style='solid', legend=True, size = 2, width=1)
# add_total_trace(fig, lst_sqrt_s, sigma_tot_log_totem_values_paper, color='gray', label='Log Totem (Paper)', line_style='solid', legend=True, size = 2, width=1)
# add_total_trace(fig, lst_sqrt_s, sigma_tot_pl_totem_values_paper, color='gray', label='PL Totem (Paper)', line_style='solid', legend=True, size = 2, width=1)

# #----

# add_data_trace(fig, x_sigma_tot_atlas, y_sigma_tot_atlas, y_error_sigma_tot_atlas, name='ATLAS', show_label=True, mode='markers')
# add_data_trace(fig, x_sigma_tot_totem, y_sigma_tot_totem, y_error_sigma_tot_totem, name='TOTEM', show_label=True, mode='markers')

# fig.update_layout(
#     title = 'σ_tot vs. √s - Ensemble Atlas and Totem in Log and PL model',
#     xaxis=dict(
#         title='√s [GeV]',
#         type='log',
#         range=[np.log10(2000), np.log10(14000)],
#     ),
#     yaxis=dict(
#         title='σ_tot [mb]',
#         range=[80, 125]
#     ),
#     showlegend=True,
#     legend=dict(
#         title='Ensembles'
#     ),
#     plot_bgcolor='white',
#     hovermode='x unified'
# )
    
# fig.update_xaxes(gridcolor='lightgray')
# fig.update_yaxes(gridcolor='lightgray')


# os.makedirs(f'../../results/sigma_tot/{save_folder}', exist_ok=True)

# fig.show(renderer="browser")
# # fig.write_html(f'../../results/sigma_tot/{save_folder}/sigma_tot_models_minimized.html')
# # fig.write_image(f'../../results/sigma_tot/{save_folder}/sigma_tot_models_minimized_zoom.pdf', width=1200, height=600)